In [0]:
dbutils.widgets.text(
    "output_table",
    "tsfrt.gsa.document_base2",
    label="source for chunk content",
)

dbutils.widgets.text(
    "vector_index_table",
    "document_base",
    label="name of pgvector index",
)

In [0]:
from pyspark.sql.functions import col, lit, concat_ws, to_json

chunk_table = dbutils.widgets.get("output_table")

# Read the table
df = spark.read.table(chunk_table)

# Select specific columns and rename them
df = df.select(
    col("id"),
    col("doc_id").alias("doc_url"),
    col("transcription").alias("content"),
    to_json(col("embedding")).alias("embedding")
).withColumn("sensitivity", lit("public"))

df.printSchema()
display(df)

In [0]:
import pandas as pd
import json
import psycopg2
from psycopg2.extras import execute_values

vector_index = dbutils.widgets.get("vector_index_table")

connection_config = dbutils.secrets.get("gsa-rag", "vector-conn-params")

pdf = df.toPandas()

for i in range(253):
    # Establish a connection to your PostgreSQL database
    data = pdf.iloc[i:i+1].copy()
    
    # Convert data types to Python native types
    data = data.astype(object).where(pd.notnull(data), None)
    
    conn = psycopg2.connect(**json.loads(connection_config))
    
    cursor = conn.cursor()
    # Prepare data for insertion (list of tuples)
    data_to_insert = [tuple(row) for row in data.itertuples(index=False, name=None)]
    # Define the SQL INSERT statement
    table_name = vector_index
    columns = ", ".join(data.columns)
    
    # Use ON CONFLICT DO UPDATE for merging (upsert)
    # Replace 'id' with your primary key or unique constraint
    insert_sql = f"""
    INSERT INTO {table_name} ({columns})
    VALUES %s
    ON CONFLICT (id) DO UPDATE SET
        content = EXCLUDED.content,
        doc_url = EXCLUDED.doc_url,
        sensitivity = EXCLUDED.sensitivity,
        embedding = EXCLUDED.embedding;
    """
    
    # Execute the bulk insertion/update
    try:
        execute_values(cursor, insert_sql, data_to_insert)
        conn.commit()
        print(f"{len(data_to_insert)} rows successfully merged into {table_name}.")
    except Exception as e:
        conn.rollback()
        print(f"{i}Error merging data: {e}")
    finally:
        cursor.close()
        conn.close()